In [2]:
!pip install underthesea 
!pip install nltk 
!pip install pyvi
!pip install gensim

In [3]:
!git clone https://github.com/NTT123/viwik18.git

Cloning into 'viwik18'...
remote: Enumerating objects: 161, done.
remote: Total 161 (delta 0), reused 0 (delta 0), pack-reused 161 (from 1)
Receiving objects: 100% (161/161), 453.04 MiB | 15.46 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (13/13), done.


In [4]:
!cat viwik18/dataset/viwik18_* > viwik18.txt

In [34]:
import nltk
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from underthesea import word_tokenize
from underthesea import sent_tokenize
from nltk.util import ngrams
from gensim.models import Word2Vec
import gensim
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
with open("viwik18.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [7]:
text[:1000]

'trang chính  internet society  internet society hay isoc là một tổ chức quốc tế hoạt động phi lợi nhuận phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành tổ chức này chú trọng đến tiêu chuẩn giáo dục và các vấn đề về chính sách với trên một bốn năm tổ chức thành viên và sáu năm không không không thành viên cá nhân isoc bao gồm những con người cụ thể trong cộng đồng internet mọi chi tiết có thể tìm thấy tại website của isoc   internet society nằm ở gần thủ đô washington dc hoa kỳ và geneva thụy sĩ số hội viên của nó bao gồm hơn một bốn năm tổ chức thành viên và hơn sáu năm không không không cá nhân thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích hiện nay tổ chức có tới chín không chi nhánh trên toàn thế giới   nhiệm vụ và mục đích hoạt động  bảo đảm cổ vũ cho sự phát triển mở rộng và sử dụng internet được thuận lợi nhất cho mọi người trên toàn thế giới   xem thêm   lịch sử internet  liên kết ngoài   isoc việt nam  trang chủ toàn cầu  

# **1. Text Preprocessing**

In [8]:
#Average word count 
word_count = len(text.split())
avg = np.mean(word_count)
print(f"The average word count is:{avg}")

The average word count is:150300503.0


In [9]:
#Check for tag patterns 
pattern = re.search(r"<[^>]+>", text)
print(bool(pattern))

False


In [10]:
#Remove URL 
text = re.sub(r"http\S+|https\S+|www\S+", "", text)

In [11]:
#Clean whitespace 
text = re.sub(r"\s+", " ",text).strip()

In [12]:
#Lowercase
text = text.lower()

In [13]:
#Tokenize the first 10000 words 
corpus = text[:10000]
tokens = []
for i in sent_tokenize(corpus):
    temp = []
    for j in word_tokenize(i):
        temp.append(j) 
    tokens.append(temp)

In [16]:
#Remove punctuation
tokens = [[t for t in sent if t.isalpha()] for sent in tokens]

In [18]:
#Remove stopwords 
with open("/kaggle/input/stop-words-in-28-languages/vietnamese.txt", "r", encoding="utf-8") as f:
    stopwords = [stopwords.strip() for stopwords in f]

stopwords = set(stopwords)
cleaned = [[w for w in sent if w not in stopwords]for sent in tokens]

# **2. Word2Vec Embedding**

**a. Using CBoW**

In [23]:
#Set up 
min_count = 1
vector_size = 100
context_size = 5
seed = 42 

In [25]:
#Build the model 
cbow = gensim.models.word2vec.Word2Vec(
    min_count=min_count,
    vector_size=vector_size,
    window=context_size,
    seed=seed)
#Build the vocabulary
cbow.build_vocab(corpus_iterable=cleaned)
#Train the model 
cbow.train(corpus_iterable=cleaned,
          total_examples=len(cleaned),
          epochs=cbow.epochs)
print(cbow.wv.vectors.shape)
print(cbow.vector_size)

(153, 100)
100


**b. Using skip-gram**

In [27]:
skipgram = gensim.models.word2vec.Word2Vec(
    min_count=min_count, 
    vector_size=vector_size, 
    window=context_size, 
    seed=seed
)
#Build the vocabulary
skipgram.build_vocab(corpus_iterable=cleaned)
#Train the model 
skipgram.train(corpus_iterable=cleaned,
              total_examples=len(cleaned),
              epochs=skipgram.epochs)
print(skipgram.wv.vectors.shape)
print(skipgram.vector_size)

(153, 100)
100


In [32]:
#Check similarity for the same query word 
print('CBoW similar to "internet":')
print(cbow.wv.most_similar("internet", topn=10))

print('\nSkip-gram similar to "internet":')
print(skipgram.wv.most_similar("internet", topn=10))

CBoW similar to "internet":
[('viết', 0.3345079720020294), ('hai', 0.23672033846378326), ('xii', 0.21625153720378876), ('chuẩn', 0.21394145488739014), ('mùa', 0.20823997259140015), ('cử', 0.2053958922624588), ('tiền', 0.2017892450094223), ('internationale', 0.20005536079406738), ('task', 0.19031672179698944), ('đầu', 0.18092425167560577)]

Skip-gram similar to "internet":
[('viết', 0.3345079720020294), ('hai', 0.23672033846378326), ('xii', 0.21625153720378876), ('chuẩn', 0.21394145488739014), ('mùa', 0.20823997259140015), ('cử', 0.2053958922624588), ('tiền', 0.2017892450094223), ('internationale', 0.20005536079406738), ('task', 0.19031672179698944), ('đầu', 0.18092425167560577)]


In [33]:
#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

# Compare embeddings for the same word
cos_cbow = cbow.wv["internet"]
cos_skip = skipgram.wv["internet"]

print("Cosine similarity between CBOW & Skip-gram 'internet':",
      cosine_similarity([cos_cbow], [cos_skip])[0][0])

Cosine similarity between CBOW & Skip-gram 'internet': 0.99999994


Since the corpus size is rather small (<10000 words) and both models are initialized with a fixed seed, the results obtained by CBoW and Skip-gram models are fairly identical. 

In [ ]:
#Visualize using PCA
def plot_embeddings(model, title):
    X = model.wv.vectors
    words = list(model.wv.index_to_key)

    # Reduce to 2D
    pca = PCA(n_components=2)
    X_reduced = pca.fit_transform(X)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(X_reduced[:, 0], X_reduced[:, 1], alpha=0.5)

    for i, word in enumerate(words[:50]):  # only show first 50 words
        plt.annotate(word, (X_reduced[i, 0], X_reduced[i, 1]))

    plt.title(title)
    plt.show()

plot_embeddings(cbow, "CBOW Embeddings (PCA)")
plot_embeddings(skipgram, "Skip-gram Embeddings (PCA)")